In [ ]:
import itk
from itk import TubeTK as tube

import vtk

import site


site.addsitedir("../../src")
from tube_viewer import tube_viewer
from tube_utils import read_group, write_group

from segment_vessels_brain_mra import  segment_vessels_brain_mra

In [5]:
img = itk.imread("../data/MRI-Cases/mra-brain.mha", itk.F)
makeIso = tube.ResampleImage.New(img, MakeHighResIso=True)
makeIso.Update()
img_iso = makeIso.GetOutput()
itk.imwrite(img_iso, "./mra_brain_iso.mha")

img_org = itk.imread("../data/MRI-Cases/mra.mha", itk.F)
makeIso = tube.ResampleImage.New(img_org, MakeHighResIso=True)
makeIso.Update()
img_org_iso = makeIso.GetOutput()
itk.imwrite(img_org_iso, "./mra_iso.mha")

In [ ]:
vess = segment_vessels_brain_mra(debug=True, debug_output_base_name="Debug/brain")
vess.set_input_image(img_iso)

In [ ]:
scales = [1.00]
vess_out = []
for scale in scales:
    vess.normalize_image(vessels_scale=scale)
    vess.identify_training_seeds()
    vess.generate_training_mask_image()
    vess.enhance_vessels()
    vess.extract_vessels(alter_image=img_org_iso) 
    vess_out.append(vess.data_vessels_group)

In [ ]:
write_group(vess_out[0],"./brain.tre")

In [ ]:
v = tube_viewer(vess_out[0])
v.render_tubes_as_surfaces()

In [ ]:
tubes = read_group("./brain.tre")
t = tube_viewer(tubes)
t.render_tubes_as_surfaces()

In [3]:
img_t2 = itk.imread("../data/MRI-Cases/mri_t2.mha", itk.F)
imReg = tube.RegisterImages[itk.Image[itk.F, 3]].New()
imReg.SetFixedImage(img_org_iso)
imReg.SetMovingImage(img_t2)
imReg.SetReportProgress(True)
imReg.SetExpectedOffsetMagnitude(20)
imReg.SetExpectedRotationMagnitude(0.01)
imReg.SetExpectedScaleMagnitude(0.01)
imReg.SetRigidMaxIterations(500)
imReg.SetRigidSamplingRatio(0.1)
imReg.SetRegistration("RIGID")
imReg.SetMetric("MATTES_MI_METRIC")
imReg.Update()
itk.imwrite(imReg.GetFinalMovingImage(), "mri_t2_reg.mha")


In [4]:
img_t1 = itk.imread("../data/MRI-Cases/mri_t1_sag.mha", itk.F)
imReg = tube.RegisterImages[itk.Image[itk.F,3]].New()
imReg.SetFixedImage(img_org_iso)
imReg.SetMovingImage(img_t1)
imReg.SetReportProgress(True)
imReg.SetExpectedOffsetMagnitude(20)
imReg.SetExpectedRotationMagnitude(0.01)
imReg.SetExpectedScaleMagnitude(0.01)
imReg.SetRigidMaxIterations(500)
imReg.SetRigidSamplingRatio(0.1)
imReg.SetRegistration("RIGID")
imReg.SetMetric("MATTES_MI_METRIC")
imReg.Update()
itk.imwrite(imReg.GetFinalMovingImage(), "mri_t1_sag_reg.mha")
